# 글로우픽 나머지 객체
    1. 생활용품 제외, v4에 있는 상품 제외 나머지 제품 파악
    2. 크롤링 진행

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from tqdm.auto import tqdm

In [2]:
fnl_df_brd_w_prd = pd.read_csv("fnl_df_brd_w_prd.csv")
fnl_df_brd_w_prd = fnl_df_brd_w_prd.iloc[:, 2:]

In [3]:
fnl_df_brd_w_prd

,brand_code,brand_name,category,product_link,product_name,naver_brd,brd_identifier,label
0,https://www.glowpick.com/brands/8016,가가알로에,['스킨케어'],['https://www.glowpick.com/products/104713'],['라율 나이트 케어 너리싱 젤'],NaN,NaN,-1
1,https://www.glowpick.com/brands/7314,가디스가든,"['스킨케어', '립메이크업', '선케어', '베이비', '클렌징']","['https://www.glowpick.com/products/96142', 'h...","['유기농 드림 리페어 선 리페어 나이트 크림', '유기농 데이 언던 선 리페어 세...",NaN,NaN,-1
2,https://www.glowpick.com/brands/10037,가란시아,"['스킨케어', '클렌징']","['https://www.glowpick.com/products/129388', '...","['쉴퓨르즈 빠뜨 뒤 마라부', '쁘쉬 마쥐끄 누벨 뽀 비쟈쥬', '렐릭시르 뒤 마...",가란시아,NaN,1
3,https://www.glowpick.com/brands/4705,가르니에,"['스킨케어', '선케어', '클렌징', '배쓰&바디', '헤어']","['https://www.glowpick.com/products/66001', 'h...","['미셀라 클렌징 워터', '프레시 아이 메이크업 리무버', '미셀라 클렌징 워터 ...",NaN,NaN,-1
4,https://www.glowpick.com/brands/10558,가온,['스킨케어'],"['https://www.glowpick.com/products/136313', '...","['새싹 담은 아이 & 페이스 크림', '씨앗담은 토닝 세럼']",NaN,NaN,-1
...,...,...,...,...,...,...,...,...
7595,https://www.glowpick.com/brands/8027,페넬로페,[],[],[],NaN,NaN,-1
7596,https://www.glowpick.com/brands/5520,퓨어체크,[],[],[],NaN,NaN,-1
7597,https://www.glowpick.com/brands/6827,하우스킨,[],[],[],NaN,NaN,-1
7598,https://www.glowpick.com/brands/6210,헬로판다,[],[],[],NaN,NaN,-1


In [43]:
for i in range(len(v4)):
    try:
        v4.at[i, '제품 수상명'] = literal_eval(v4.loc[i, '제품 수상명'])
        v4.at[i, '제품 수상 부문'] = literal_eval(v4.loc[i, '제품 수상 부문'])
        v4.at[i, '제품 수상 순위'] = literal_eval(v4.loc[i, '제품 수상 순위'])
        v4.at[i, '제품 판매처'] = literal_eval(v4.loc[i, '제품 판매처'])
        v4.at[i, '제품 성분구성: 위험도 미정 (성분(한글))'] = literal_eval(v4.loc[i, '제품 성분구성: 위험도 미정 (성분(한글))'])
        v4.at[i, '제품 성분구성: 위험도 미정 (성분(영어))'] = literal_eval(v4.loc[i, '제품 성분구성: 위험도 미정 (성분(영어))'])
        v4.at[i, '제품 성분구성: 위험도 미정 (성분(설명))'] = literal_eval(v4.loc[i, '제품 성분구성: 위험도 미정 (성분(설명))'])
        v4.at[i, '제품 성분구성: 낮은 위험도 (성분(한글))'] = literal_eval(v4.loc[i, '제품 성분구성: 낮은 위험도 (성분(한글))'])
        v4.at[i, '제품 성분구성: 낮은 위험도 (성분(영어))'] = literal_eval(v4.loc[i, '제품 성분구성: 낮은 위험도 (성분(영어))'])
        v4.at[i, '제품 성분구성: 낮은 위험도 (성분(설명))'] = literal_eval(v4.loc[i, '제품 성분구성: 낮은 위험도 (성분(설명))'])
        v4.at[i, '제품 성분구성: 중간 위험도 (성분(한글))'] = literal_eval(v4.loc[i, '제품 성분구성: 중간 위험도 (성분(한글))'])
        v4.at[i, '제품 성분구성: 중간 위험도(성분(영어))'] = literal_eval(v4.loc[i, '제품 성분구성: 중간 위험도(성분(영어))'])
        v4.at[i, '제품 성분구성: 중간 위험도 (성분(설명))'] = literal_eval(v4.loc[i, '제품 성분구성: 중간 위험도 (성분(설명))'])
        v4.at[i, '제품 성분구성: 높은 위험도 (성분(한글))'] = literal_eval(v4.loc[i, '제품 성분구성: 높은 위험도 (성분(한글))'])
        v4.at[i, '제품 성분구성: 높은 위험도 (성분(영어))'] = literal_eval(v4.loc[i, '제품 성분구성: 높은 위험도 (성분(영어))'])
        v4.at[i, '제품 성분구성: 높은 위험도 (성분(설명))'] = literal_eval(v4.loc[i, '제품 성분구성: 높은 위험도 (성분(설명))'])
        v4.at[i, '제품 카테고리'] = literal_eval(v4.loc[i, '제품 카테고리'])
        
    except:
        pass

In [4]:
v4 = pd.read_csv("v4_update.csv")
v4 = v4.iloc[:, 1:]

In [5]:
len(v4)

64287

In [6]:
lst_v4_prd_link = list(v4.loc[:, '제품 link'])

In [19]:
len(list(set(lst_v4_prd_link)))

64167

In [7]:
for i in range(len(fnl_df_brd_w_prd)):
    try: 
        fnl_df_brd_w_prd.at[i, 'product_link'] = literal_eval(fnl_df_brd_w_prd.loc[i, 'product_link'])
        fnl_df_brd_w_prd.at[i, 'product_name'] = literal_eval(fnl_df_brd_w_prd.loc[i, 'product_name'])
    except:
        pass

In [8]:
total_cnt = 0

for i in range(len(fnl_df_brd_w_prd)):
    total_cnt += len(fnl_df_brd_w_prd.loc[i, 'product_link'])

In [15]:
fnl_df_brd_w_prd_prd_lst = []

for i in tqdm(range(len(fnl_df_brd_w_prd))):
    for j in range(len(fnl_df_brd_w_prd.loc[i, 'product_link'])):
        fnl_df_brd_w_prd_prd_lst.append(fnl_df_brd_w_prd.loc[i, 'product_link'][j])

  0%|          | 0/7600 [00:00<?, ?it/s]

In [17]:
len(list(set(fnl_df_brd_w_prd_prd_lst)))

111749

In [9]:
total_cnt #전체 제품 개수 

112253

In [11]:
lst_cr = []

In [12]:
for i in tqdm(range(len(fnl_df_brd_w_prd))):
    brd_idx = i
    for j in range(len(fnl_df_brd_w_prd.loc[i, 'product_link'])):
        if fnl_df_brd_w_prd.loc[i, 'product_link'][j] not in lst_v4_prd_link:
            lst_cr.append(fnl_df_brd_w_prd.loc[i, 'product_link'][j])

  0%|          | 0/7600 [00:00<?, ?it/s]

In [13]:
len(lst_cr)

47687

In [256]:
import pickle

with open("lst_glowpick_crawling_link.txt", "wb") as f:
    pickle.dump(fnl_minus_v4_lst, f)

In [181]:
fnl_df_brd_w_prd.loc[7598, 'product_link']

[]

In [20]:
v4_not_df = []

for i in tqdm(range(len(v4))):
    if v4.loc[i, '제품 link'] not in fnl_df_brd_w_prd_prd_lst:
        v4_not_df.append(v4.loc[i, '제품 link'])

  0%|          | 0/64287 [00:00<?, ?it/s]

In [22]:
len(v4_not_df)

94

In [255]:
fnl_minus_v4_lst = []

for i in tqdm(range(len(fnl_df_brd_w_prd))):
    for j in range(len(fnl_df_brd_w_prd.loc[i, 'product_link'])):
        if fnl_df_brd_w_prd.loc[i, 'product_link'][j] not in lst_v4_prd_link:
            fnl_minus_v4_lst.append(fnl_df_brd_w_prd.loc[i, 'product_link'][j])

  0%|          | 0/7600 [00:00<?, ?it/s]

In [248]:
len(fnl_minus_v4_lst)

47687

In [253]:
112253 - 111974

279

In [252]:
47687 + 64287

111974

In [231]:
a_sub_b = [x for x in missing if x not in lst_cr]

In [232]:
len(a_sub_b)

['쉴퓨르즈 빠뜨 뒤 마라부',
 '쁘쉬 마쥐끄 누벨 뽀 비쟈쥬',
 '렐릭시르 뒤 마라부',
 '쑤르쓰 미쎌레르 앙샹테 로즈 당탕',
 '미스테리유 콘센트레이트 세럼',
 '미스테리유 밀 에 엉 쥬르 크렘 드 쥬르',
 '디아볼리끄 토마트 리쉬',
 '디아볼리끄 토마트 크렘 도',
 '아쿠아 레브 뛰',
 '겔라비트 무어 비타민 크림',
 '겔라비트 무어 비타민 크림 AF',
 '겔라산 핸드크림',
 '메드 데오드란트 풋 크림',
 '리치 모이스춰라이징 크림',
 '워터',
 '모이스춰라이징 크림 마스크',
 '프로텍티브 샤워젤',
 '뉴트리션 립 밤 [스틱]',
 '나이트 세럼',
 '젠틀 수딩 로션',
 '퓨리파잉 포밍 워터',
 '뉴트리션 인텐스 립 밤 [튜브]',
 '화이트 세럼 에끌라',
 '젠틀 아이메이크업 리무버',
 '퓨어 페이셜 젤',
 '화이트 로션 에끌라',
 '모이스춰라이징 엘릭시어',
 '모이스춰라이징 로션',
 '토닝 로션 - 유칼립투스',
 '샴푸 두쉐르',
 '울트라 리치 모이스춰라이징 크림',
 '쏠레흐 오뜨 프로떽시옹 [SPF50/PA+++]',
 '소프트 바디 스크럽',
 '모이스춰라이징 라이트 플루이드',
 '뉴트리션 인텐스 젠틀 마스크',
 '젤 이지엔 앙띰',
 '젠틀 컨디셔너 [애프리콧씨드]',
 '스티뮬레이팅 로션',
 '바이탈리티 샴푸 [오렌지]',
 '리페어링 풋 크림',
 '너리싱 샴푸 [아카시아허니]',
 '글로잉 샴푸 [알로에베라]',
 '그린티 수딩 데오도란트',
 '쉐이빙 크림 델리키트 스킨&비어드',
 '소프트 페이셜 스크럽',
 '뉴트리션 인텐스 크림',
 '바디 밀크',
 '엑스폴리에이팅 풋 젤 스크럽',
 '화이트 스웽 에끌라 [데이 크림]',
 '화이트 스웽 에끌라 [나이트 크림]',
 '퓨리파잉&코렉팅 플루이드',
 '퓨리파잉 샴푸 [케이드 우드]',
 '토닝 샴푸 [페퍼민트]',
 '젠틀 클렌징 밀크',
 '애프터 쉐이브 모이스춰라이징 밤',
 '아이즈&립스 플루이드 컨투어',
 '데이&나이

In [230]:
missing - lst_cr

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [208]:
missing[2]

'렐릭시르 뒤 마라부'

In [219]:
v4 = pd.read_csv("v4_update.csv")
v4 = v4.iloc[:, 1:]

In [218]:
v4.iloc[67]

index                                                                      67
브랜드 link                                                                  NaN
브랜드명                                                                   닥터브로너스
브랜드 id                                                                    NaN
제품 link                              https://www.glowpick.com/products/150273
제품 id                                                                  150273
제품명                                                              티트리 퓨어 캐스틸 솝
제품 용량                                                                   475ml
제품 가격                                                                 20,000원
유사 제품군 랭킹(int)                                                             8위
유사 제품군 랭킹(text)                                                           바디솝
브랜드 내 랭킹(int)                                                              3위
제품 수상명                                                          